In [1]:
import json
import torch
import pickle
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from transformers import AdamW, GPT2Config, GPT2LMHeadModel

In [2]:
json_wikipedia_file = 'text/AA/wiki_00' 
plain_wikipedia_file = 'plain_wikipedia.txt'

Read the json file and write the article title and text into a single text file

In [3]:
with open(plain_wikipedia_file, 'w') as f:
    for line in open(json_wikipedia_file, 'r', encoding='utf-8'):
        article = json.loads(line)
        f.write(article['title'])
        f.write('\n')
        f.write(article['text'])
        f.write('\n')

Prepare the Tokenizer: it is the object that will analyse all the text and build the vocabulary

In [4]:
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"], vocab_size=5000)

Analyse the text using the tikenizer to get the vocabulary

In [5]:
tokenizer.train(files=[plain_wikipedia_file], trainer=trainer)

In [6]:
sentence = "ប្រាសាទ អង្គរវត្ត"

output = tokenizer.encode(sentence)
print(output.ids)
print(output.tokens)

[3632, 4960, 1138, 3391]
['ប្រាសាទ', ' អង្គ', 'រ', 'វត្ត']


In [7]:
print(tokenizer.decode(output.ids, skip_special_tokens=False))

ប្រាសាទ  អង្គ រ វត្ត


In [8]:
max_length = 64
batch_size = 32
padding_value = tokenizer.token_to_id("[PAD]")
n_layer=4
n_head=4
n_embed=768

In [9]:
tokenized_texts = []
with open(plain_wikipedia_file, 'r', encoding='utf-8') as file:
    for line in tqdm(file):
        tokenized_line = tokenizer.encode(line.strip()).ids
        if tokenized_line:
            tokenized_texts.append(tokenized_line[:max_length])

170392it [00:10, 15944.27it/s]


In [10]:
class TextDataset(Dataset):
    def __init__(self, tokenized_texts):
        self.tokenized_texts = tokenized_texts

    def __len__(self):
        return len(self.tokenized_texts)

    def __getitem__(self, idx):
        return torch.tensor(self.tokenized_texts[idx])

def collate_batch(batch):
    # Pad the sequences in the batch
    batch_padded = pad_sequence([sequence for sequence in batch], 
                                batch_first=True, padding_value=padding_value)
    # Create attention masks
    attention_masks = torch.zeros(batch_padded.shape, dtype=torch.long)
    attention_masks[batch_padded != padding_value] = 1

    return batch_padded, attention_masks


dataset = TextDataset(tokenized_texts)
data_loader = DataLoader(dataset, batch_size=128, shuffle=True, collate_fn=collate_batch)

In [11]:
config = GPT2Config(
    vocab_size = len(tokenizer.get_vocab()), 
    n_positions = max_length, 
    n_layer = n_layer, 
    n_head = n_head, 
    n_embed = n_embed, 
    pad_token_id = padding_value)
model = GPT2LMHeadModel(config)
device = torch.device("cuda")
model.to(device)

# Set up optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [12]:
model.save_pretrained("gpt2-wikipedia-khmer-no-pretrain")

In [12]:
sentence_to_complete = "ប្រាសាទ អង្គរវត្ត"
input_text = sentence_to_complete
input_ids = torch.tensor([tokenizer.encode(input_text).ids]).to(device)
attention_mask = torch.ones_like(input_ids).to(device)
model.eval()
output = model.generate(
    input_ids, 
    attention_mask=attention_mask, 
    max_length=max_length, 
    num_beams=1, 
    num_return_sequences=1
    ).to(device)
for prediction in output:
    predicted_text = tokenizer.decode(prediction.tolist(), skip_special_tokens=True)
    print("Predicted text:", predicted_text)

Predicted text: ប្រាសាទ  អង្គ រ វត្ត េយ្យ េយ្យ េយ្យ េយ្យ េយ្យ េយ្យ េយ្យ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ ប្តូរ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ និ


In [13]:
batch_id = 0
max_epoch = 20
for epoch in range(max_epoch):
    model.train()
    for batch, attention_mask in data_loader:
        optimizer.zero_grad()
        input_ids = batch.to(device)
        attention_mask = attention_mask.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=input_ids)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        if batch_id % 1000 == 0:
            print(f'Epoch: {epoch}, Batch: {batch_id}/{len(data_loader)*max_epoch}, Loss: {loss.item()}')
        batch_id += 1
        # if batch_id % 500 == 0:
        #     break
    model.eval()
    sentence_to_complete = "ប្រាសាទ អង្គរវត្ត"
    input_text = sentence_to_complete
    input_ids = torch.tensor([tokenizer.encode(input_text).ids]).to(device)
    attention_mask = torch.ones_like(input_ids).to(device)
    output = model.generate(
        input_ids, 
        attention_mask = attention_mask, 
        max_length = max_length, 
        num_beams = 1, 
        num_return_sequences = 1).to(device)
    for prediction in output:
        predicted_text = tokenizer.decode(prediction.tolist(), skip_special_tokens=True)
        print("Predicted text:", predicted_text)

Epoch: 0, Batch: 0/25940, Loss: 8.236664772033691
Epoch: 0, Batch: 1000/25940, Loss: 2.305311679840088
Predicted text: ប្រាសាទ  អង្គ រ វត្ត
Epoch: 1, Batch: 2000/25940, Loss: 2.2689266204833984
Predicted text: ប្រាសាទ  អង្គ រ វត្ត
Epoch: 2, Batch: 3000/25940, Loss: 2.1256203651428223
Predicted text: ប្រាសាទ  អង្គ រ វត្ត
Epoch: 3, Batch: 4000/25940, Loss: 1.6820898056030273
Epoch: 3, Batch: 5000/25940, Loss: 1.8167405128479004
Predicted text: ប្រាសាទ  អង្គ រ វត្ត
Epoch: 4, Batch: 6000/25940, Loss: 1.7585417032241821
Predicted text: ប្រាសាទ  អង្គ រ វត្ត ប្រាសាទ ព្រះ វិហារ វត្ត ព្រះ វិហារ វត្ត ព្រះ វិហារ វត្ត ព្រះ វិហារ ព្រះ វិហារ វត្ត ព្រះ វិហារ វត្ត ព្រះ វិហារ វត្ត ព្រះ វិហារ ព្រះ វិហារ វត្ត ព្រះ វិហារ វត្ត ព្រះ វិហារ វត្ត ព្រះ វិហារ វត្ត ព្រះ វិហារ វត្ត ព្រះ វិហារ ព្រះ វិហារ វត្ត ព្រះ វិហារ វត្ត ព្រះ វិហារ ព្រះ វិហារ វត្ត ព្រះ
Epoch: 5, Batch: 7000/25940, Loss: 1.5956480503082275
Predicted text: ប្រាសាទ  អង្គ រ វត្ត ប្រាសាទ ព្រះ វិហារ ព្រះ ព្រះ វិហារ ព្រះ ព្រះ វិហារ ព្រះ ព្រះ វិហារ ព្រ

In [14]:
model.save_pretrained("gpt2-wikipedia-khmer")

In [11]:
model = GPT2LMHeadModel.from_pretrained("gpt2-wikipedia-khmer")

In [7]:
outputs = json.load(open('outputs.json', 'r'))

In [15]:
sentence_to_complete = "ប្រាសាទ អង្គរវត្ត"
input_text = sentence_to_complete
input_ids = torch.tensor([tokenizer.encode(input_text).ids])
attention_mask = torch.ones_like(input_ids)
model.train()
output = model.generate(
    input_ids, 
    attention_mask=attention_mask, 
    max_length=64, 
    num_beams=10,
    num_return_sequences=10
    )
for prediction in output:
    predicted_text = tokenizer.decode(prediction.tolist(), skip_special_tokens=True)
    print("Predicted text:", predicted_text)

Predicted text: ប្រាសាទ  អង្គ រ វត្ត រ ជ្ជ កាល ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧  ( សំ ស្ ក្រ ឹត )  ( ប្រ .ស | គ.ស   ០០ ០០ - ១០ ៥០ )  រ ជ្ជ កាល គ្រ ង រាជ  ( គ.ស  ១ ០០ ៦ - ១ ០០ ១ - ១ ០០ ១ )  រ ជ្ជ កាល គ្រ ង រាជ
Predicted text: ប្រាសាទ  អង្គ រ វត្ត រ ជ្ជ កាល ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧  ( សំ ស្ ក្រ ឹត )  ( ប្រ .ស | គ.ស   ០០ ០០ - ១០ ៥០ )  រ ជ្ជ កាល គ្រ ង រាជ  ( គ.ស  ១ ០០ ៦ - ១ ០០ ១ - ១ ០០ ១ )  ក្រោយ គ្រ ង រាជ សម្បត្តិ នៅ
Predicted text: ប្រាសាទ  អង្គ រ វត្ត រ ជ្ជ កាល ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧  ( សំ ស្ ក្រ ឹត )  ( ប្រ .ស | គ.ស   ០០ ០០ - ១០ ៥០ )  រ ជ្ជ កាល គ្រ ង រាជ  ( គ.ស  ១ ០០ ៦ - ១ ០០ ១ - ១ ០០ ១ )  ក្រោយ គ្រ ង រាជ សម្បត្តិ បន្ត
Predicted text: ប្រាសាទ  អង្គ រ វត្ត រ ជ្ជ កាល ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧  ( សំ ស្ ក្រ ឹត )  ( ប្រ .ស | គ.ស   ០០ ០០ - ១០ ៥០ )  រ ជ្ជ កាល គ្រ ង រាជ  ( គ.ស  ១ ០០ ៦ - ១ ០០ ១ - ១ ០០ ១ )  ក្រោយ គ្រ ង រាជ សម្បត្តិ ក្នុង
Predicted text: ប្រាសាទ  អង្គ រ វត្ត រ ជ្ជ កាល ព្រះបាទ ជ័យ វរ្ម័ន ទី ៧  ( សំ ស្ ក្រ ឹត )  ( ប្រ .ស | គ.ស   ០០ ០០ - ១០ ៥០ )  រ ជ្ជ កាល គ្រ ង រាជ  ( គ.ស  ១ ០០ ៦ - ១ ០០ ១ - ១